In [429]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [230]:
url1 = '2023_LoL_esports_match_data_from_OraclesElixir.csv'
url2 = '2024_LoL_esports_match_data_from_OraclesElixir.csv'


In [231]:
df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)
print(df1.shape, df2.shape)
df = pd.concat([df1, df2])

C:\Users\dskon\AppData\Local\Temp\ipykernel_33064\399056814.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(url1)


(130176, 123) (8508, 123)


# Constants/Lists

In [232]:
df = df[df['position'] != 'team']


In [233]:
region_d = {'LEC':0, 'LCS':1,'LPL':2,'LCK':3}
pos_d = {'top':0,'jng':1,'mid':2,'bot':3,'sup':4}
side = {'Blue':0, 'Red':1}

In [270]:
majorleagues = ['LCK','LPL','LCS','LEC'] #LPLLOL? its portugeuse LOL
t2leagues = ['CBLOL','LAS','LCO','LFL','LJL','LLA','PCS','VCS','TCL']
intl = ['MSI','WLDs']
west = ['LCS','LEC']


In [235]:
#cols not relevant
dropcols = ['gameid','datacompleteness', 'url', 'year', 'split', 'date', 'patch','playerid','teamname','playername','ban1','ban3','ban4','ban5','ban2','doublekills','triplekills','quadrakills','pentakills',]
#basic info
basic = 'league, patch, champion, position, result, kills, deaths, assists, dragons, firstdragon, firstbloodkill, chemtechs, dragons, playername'
basic = basic.split(', ')

#MISSING?
missing = ['hextechs', 'opp_turretplates', 'turretplates', 'firsttothreetowers',
       'firstmidtower', 'opp_towers', 'towers', 'firsttower', 'firstbaron',
       'opp_heralds', 'heralds', 'monsterkillsenemyjungle', 'firstherald',
       'opp_elders', 'elders', 'dragons (type unknown)', 'firstdragon',
       'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes',
       'infernals', 'monsterkillsownjungle', 'clouds', 'oceans', 'chemtechs',
       'gspd', 'mountains']

# Data Exploration

In [269]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,131.0,322.0,263.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,117.0,-357.0,-1323.0,-43.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,162.0,-479.0,-324.0,-26.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,122.0,200.0,292.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0
4,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,3.0,-216.0,-579.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [244]:
expl = df.copy()
expl = expl[basic]
expl = expl[expl['patch'].isna()]
expl.league

54540    LPL
54541    LPL
54542    LPL
54543    LPL
54544    LPL
        ... 
61325    LDL
61326    LDL
61327    LDL
61328    LDL
61329    LDL
Name: league, Length: 100, dtype: object

In [238]:
dfn = df.drop(dropcols, axis = 1).drop(missing, axis = 1)
dfn = dfn[dfn['league'].isin(west)]
temp = ['participantid','teamid']
dfn = dfn.drop(temp, axis = 1)
print(dfn.columns.tolist())
print(dfn.shape)
dfn = dfn.reset_index(drop = True)
dfn.dtypes.value_counts()

['league', 'playoffs', 'game', 'side', 'position', 'champion', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'barons', 'opp_barons', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dpm', 'damageshare', 'damagetakenperminute', 'damagemitigatedperminute', 'wardsplaced', 'wpm', 'wardskilled', 'wcpm', 'controlwardsbought', 'visionscore', 'vspm', 'totalgold', 'earnedgold', 'earned gpm', 'earnedgoldshare', 'goldspent', 'total cs', 'minionkills', 'monsterkills', 'cspm', 'goldat10', 'xpat10', 'csat10', 'opp_goldat10', 'opp_xpat10', 'opp_csat10', 'golddiffat10', 'xpdiffat10', 'csdiffat10', 'killsat10', 'assistsat10', 'deathsat10', 'opp_killsat10', 'opp_assistsat10', 'opp_deathsat10', 'goldat15', 'xpat15', 'csat15', 'opp_goldat15', 'opp_xpat15', 'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15', 'assistsat15', 'deathsat15', 'opp_killsat15', 'opp

float64    52
int64      18
object      4
Name: count, dtype: int64

# Complete model

In [239]:

def encode_lspc(df):
    temp = df.copy()
    encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
    enc_league = encoder.fit_transform(df.league.values.reshape(-1,1))
    enc_side = encoder.fit_transform(df.side.values.reshape(-1,1))
    enc_pos = encoder.fit_transform(df.position.values.reshape(-1,1))
    enc_champ = encoder.fit_transform(df.champion.values.reshape(-1,1))
    #print(enc_champ.sum())
    temp = temp.drop(['league','side','position','champion'],axis = 1)
    return pd.concat([temp,enc_league, enc_side,enc_pos,enc_champ],axis = 1)

In [243]:
dfn1 = dfn.copy()
dfn1 = encode_lspc(dfn1)
dfn1.dtypes

playoffs        int64
game            int64
gamelength      int64
result          int64
kills           int64
               ...   
x0_Zeri       float64
x0_Ziggs      float64
x0_Zilean     float64
x0_Zoe        float64
x0_Zyra       float64
Length: 218, dtype: object

In [248]:
y = dfn1.result
X = dfn1.drop(['result'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [267]:
reg = LogisticRegression(max_iter=10000)
reg.fit(X_train,y_train)

C:\Users\dskon\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=10000)

In [268]:
res = reg.predict(X_test)
score = reg.score(X_test, y_test)
score

0.9724545007378259

# Comp model

In [336]:
dfc = df[df['league'].isin(majorleagues)]
dfc = dfc[['gameid','champion','result','side','position']]
comps = dfc.groupby(['gameid','result']).champion.unique()
df3 = pd.DataFrame(comps).reset_index().rename({'champion':'comp'},axis = 1)
df3[df3['gameid'] == '9691-9691_game_1']
df3.head(5)

,gameid,result,comp
0,10005-10005_game_1,0,"[Sion, Vi, Ryze, Aphelios, Nautilus]"
1,10005-10005_game_1,1,"[Kennen, Wukong, Annie, Varus, Leona]"
2,10005-10005_game_2,0,"[Gragas, Sejuani, Yone, Aphelios, Nautilus]"
3,10005-10005_game_2,1,"[Kennen, Wukong, LeBlanc, Jinx, Leona]"
4,10005-10005_game_3,0,"[K'Sante, Xin Zhao, LeBlanc, Zeri, Rakan]"


In [337]:
dfc.head(6)

,gameid,champion,result,side,position
204,9691-9691_game_1,K'Sante,0,Blue,top
205,9691-9691_game_1,Vi,0,Blue,jng
206,9691-9691_game_1,Akali,0,Blue,mid
207,9691-9691_game_1,Caitlyn,0,Blue,bot
208,9691-9691_game_1,Lux,0,Blue,sup
209,9691-9691_game_1,Renekton,1,Red,top


In [471]:
dfinv = df3.copy()
dfinv['result'] = (dfinv['result'] == False)*1
dfinv = dfinv.rename({'comp':'ecomp'}, axis = 1)

In [472]:
dfc = dfc.reset_index(drop = True)
df_temp = pd.merge(dfc,df3 , on=["gameid",'result'], how="left")
df_temp.head(5)

,gameid,champion,result,side,position,comp
0,9691-9691_game_1,K'Sante,0,Blue,top,"[K'Sante, Vi, Akali, Caitlyn, Lux]"
1,9691-9691_game_1,Vi,0,Blue,jng,"[K'Sante, Vi, Akali, Caitlyn, Lux]"
2,9691-9691_game_1,Akali,0,Blue,mid,"[K'Sante, Vi, Akali, Caitlyn, Lux]"
3,9691-9691_game_1,Caitlyn,0,Blue,bot,"[K'Sante, Vi, Akali, Caitlyn, Lux]"
4,9691-9691_game_1,Lux,0,Blue,sup,"[K'Sante, Vi, Akali, Caitlyn, Lux]"


In [473]:
def foo(row):
    l = row['comp']
    ch = row['champion']
    index = np.argwhere(l==ch)
    return np.delete(l, index)

df_full = pd.merge(df_temp, dfinv, on=['gameid','result'], how = 'left').drop(['gameid'],axis=1)
#df_full['comp'] = df_full.apply(foo, axis=1)
df_full

,champion,result,side,position,comp,ecomp
0,K'Sante,0,Blue,top,"[K'Sante, Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]"
1,Vi,0,Blue,jng,"[K'Sante, Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]"
2,Akali,0,Blue,mid,"[K'Sante, Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]"
3,Caitlyn,0,Blue,bot,"[K'Sante, Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]"
4,Lux,0,Blue,sup,"[K'Sante, Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]"
...,...,...,...,...,...,...
19825,Olaf,1,Red,top,"[Olaf, Maokai, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]"
19826,Maokai,1,Red,jng,"[Olaf, Maokai, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]"
19827,Azir,1,Red,mid,"[Olaf, Maokai, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]"
19828,Varus,1,Red,bot,"[Olaf, Maokai, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]"


In [436]:
df_full

,champion,result,side,position,comp,ecomp,ally1,ally2,ally3,ally4,en1,en2,en3,en4,en5
0,K'Sante,0,Blue,top,"[Vi, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]",Vi,Akali,Caitlyn,Lux,Renekton,Maokai,Azir,Kalista,Nautilus
1,Vi,0,Blue,jng,"[K'Sante, Akali, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]",K'Sante,Akali,Caitlyn,Lux,Renekton,Maokai,Azir,Kalista,Nautilus
2,Akali,0,Blue,mid,"[K'Sante, Vi, Caitlyn, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]",K'Sante,Vi,Caitlyn,Lux,Renekton,Maokai,Azir,Kalista,Nautilus
3,Caitlyn,0,Blue,bot,"[K'Sante, Vi, Akali, Lux]","[Renekton, Maokai, Azir, Kalista, Nautilus]",K'Sante,Vi,Akali,Lux,Renekton,Maokai,Azir,Kalista,Nautilus
4,Lux,0,Blue,sup,"[K'Sante, Vi, Akali, Caitlyn]","[Renekton, Maokai, Azir, Kalista, Nautilus]",K'Sante,Vi,Akali,Caitlyn,Renekton,Maokai,Azir,Kalista,Nautilus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19825,Olaf,1,Red,top,"[Maokai, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]",Maokai,Azir,Varus,Neeko,Aatrox,Viego,Sylas,Kalista,Renata Glasc
19826,Maokai,1,Red,jng,"[Olaf, Azir, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]",Olaf,Azir,Varus,Neeko,Aatrox,Viego,Sylas,Kalista,Renata Glasc
19827,Azir,1,Red,mid,"[Olaf, Maokai, Varus, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]",Olaf,Maokai,Varus,Neeko,Aatrox,Viego,Sylas,Kalista,Renata Glasc
19828,Varus,1,Red,bot,"[Olaf, Maokai, Azir, Neeko]","[Aatrox, Viego, Sylas, Kalista, Renata Glasc]",Olaf,Maokai,Azir,Neeko,Aatrox,Viego,Sylas,Kalista,Renata Glasc


In [482]:
def team(df):
    df[['ally1','ally2','ally3','ally4']] = pd.DataFrame(df.comp.tolist(), index=df.index)
    df[['en1','en2','en3','en4','en5']] = pd.DataFrame(df.ecomp.tolist(), index=df.index)
    return df
df_lfinal = team(df_full).drop(['champion','comp','ecomp','position','side'],axis = 1)
df_lfinal

ValueError: Columns must be same length as key

In [477]:
def encode_lspc1(df):
    temp = df.copy()
    encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
    lst = [ 'ally1', 'ally2', 'ally3', 'ally4',
        'en1', 'en2', 'en3', 'en4', 'en5']
    for i in range(len(lst)):
        enc = encoder.fit_transform(df[lst[i]].values.reshape(-1,1))
        temp = pd.concat([temp,enc],axis = 1).drop([lst[i]],axis=1)
    return temp

def en(df,col):
    temp = df.copy()
    enc = OrdinalEncoder()
    enco = pd.DataFrame(enc.fit_transform(df[col].values.tolist()).tolist())
    return pd.concat([temp,enco],axis=1).drop([col],axis=1)
df_full.comp.values.tolist()
df_asdf = en(df_full,'comp').rename({0:'z',1:'x',2:'c',3:'v',4:'b'},axis=1)
df_asdf = en(df_asdf, 'ecomp').rename({0:'a',1:'s',2:'d',3:'f',4:'g'},axis=1)
encoder = OneHotEncoder(sparse_output=False).set_output(transform="pandas")
enc = encoder.fit_transform(df_asdf.side.values.reshape(-1,1))
df_asdf = pd.concat([df_asdf,enc],axis=1).drop(['side'],axis=1)
enc = encoder.fit_transform(df_asdf.champion.values.reshape(-1,1))
df_asdf = pd.concat([df_asdf,enc],axis=1).drop(['champion','position'],axis=1)
df_asdf

,result,z,x,c,v,b,a,s,d,f,...,x0_Yasuo,x0_Yone,x0_Yuumi,x0_Zac,x0_Zed,x0_Zeri,x0_Ziggs,x0_Zilean,x0_Zoe,x0_Zyra
0,0,19.0,36.0,1.0,2.0,21.0,36.0,18.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,19.0,36.0,1.0,2.0,21.0,36.0,18.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,19.0,36.0,1.0,2.0,21.0,36.0,18.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,19.0,36.0,1.0,2.0,21.0,36.0,18.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,19.0,36.0,1.0,2.0,21.0,36.0,18.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19825,1,31.0,18.0,5.0,23.0,28.0,0.0,37.0,38.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19826,1,31.0,18.0,5.0,23.0,28.0,0.0,37.0,38.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19827,1,31.0,18.0,5.0,23.0,28.0,0.0,37.0,38.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19828,1,31.0,18.0,5.0,23.0,28.0,0.0,37.0,38.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [480]:
y = df_comps.result
X = df_comps.drop(['result'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
reg = LogisticRegression(max_iter=1000000)
reg.fit(X_train,y_train)

LogisticRegression(max_iter=1000000)

In [481]:
res = reg.predict(X_test)
score = reg.score(X_test, y_test)
score

0.5997860635696821

# Year 2023 Games

In [4]:
df1.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,131.0,322.0,263.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,117.0,-357.0,-1323.0,-43.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,162.0,-479.0,-324.0,-26.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,122.0,200.0,292.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0
4,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,...,3.0,-216.0,-579.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [6]:
df1.groupby('league').league.unique()

league
AL                    [AL]
CBLOL              [CBLOL]
CBLOLA            [CBLOLA]
CDF                  [CDF]
CT                    [CT]
DDH                  [DDH]
EBL                  [EBL]
EL                    [EL]
EM                    [EM]
EPL                  [EPL]
ESLOL              [ESLOL]
GL                    [GL]
GLL                  [GLL]
HC                    [HC]
HM                    [HM]
IC                    [IC]
LAS                  [LAS]
LCK                  [LCK]
LCKC                [LCKC]
LCO                  [LCO]
LCS                  [LCS]
LDL                  [LDL]
LEC                  [LEC]
LFL                  [LFL]
LFL2                [LFL2]
LHE                  [LHE]
LJL                  [LJL]
LJLA                [LJLA]
LLA                  [LLA]
LMF                  [LMF]
LPL                  [LPL]
LPLOL              [LPLOL]
LRN                  [LRN]
LRS                  [LRS]
LVP SL            [LVP SL]
MSI                  [MSI]
NACL                [

# Year 2024 Games

In [5]:
df2.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10660-10660_game_1,partial,https://lpl.qq.com/es/stats.shtml?bmid=10660,DCup,2023,NaN,0,2024-01-01 05:13:15,1,13.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
